# Соединения

Нередко возникает ситуация, когда нам надо получить данные из нескольких таблиц одновременно. Для этого используется специальный класс операций, называемый соединения (join).

Рассмотрим 2 корзины с товарами:

basket_a:

| id | fruit |
|---|---|
| 1 | 'Apple' |
| 2 | 'Orange' |
| 3 | 'Banana' |
| 4 | 'Cucumber' |
 
basket_b:

| id | fruit |
|---|---|
| 1 | 'Orange' |
| 2 | 'Apple' |
| 3 | 'Watermelon' |
| 4 | 'Pear' |

## PostgreSQL inner join

Рассмотрим запрос, соединяющий данные из первой и второй таблицы, используя значения колонки fruit:

```sql
SELECT
    a.id id_a,
    a.fruit fruit_a,
    b.id id_b,
    b.fruit fruit_b
FROM
    basket_a a
INNER JOIN basket_b b ON a.fruit = b.fruit;
```

| id_a | fruit_d | id_a | fruit_d |
|---|---|---|---|
| 1 | 'Apple' | 2 | 'Apple' |
| 2 | 'Orange' | 1 | 'Orange' |

Легко видеть, что inner join возвращает данные по строкам, содержащим одинаковые значения. Если смотреть на таблицы как на множества строк, то результат их выполнения можно представить на следующей диаграмме Вена:

![](PostgreSQL-Join-Inner-Join.png)

## PostgreSQL left join

```sql
SELECT
    a.id id_a,
    a.fruit fruit_a,
    b.id id_b,
    b.fruit fruit_b
FROM
    basket_a a
LEFT JOIN basket_b b ON a.fruit = b.fruit;
```

| id_a | fruit_d | id_a | fruit_d |
|---|---|---|---|
| 1 | 'Apple' | 2 | 'Apple' |
| 2 | 'Orange' | 1 | 'Orange' |
| 3 | 'Banana' | _(null)_ | _(null)_ |
| 4 | 'Cucumber' | _(null)_ | _(null)_ |

left join возвращает все данные из первой таблицы. Если по ним есть совпадения в правой - то они обогащаются сотвествующими данными, иначе туда записывается специальное значение - NULL

![](PostgreSQL-Join-Left-Join.png)

## PostgreSQL right join

Это обратная версия left join'а. Рассмотрим пример:

```sql
SELECT
    a.id id_a,
    a.fruit fruit_a,
    b.id id_b,
    b.fruit fruit_b
FROM
    basket_a a
RIGHT JOIN basket_b b ON a.fruit = b.fruit;
```

| id_a | fruit_d | id_a | fruit_d |
|---|---|---|---|
| 1 | 'Apple' | 2 | 'Apple' |
| 2 | 'Orange' | 1 | 'Orange' |
| _(null)_ | _(null)_ | 3 | 'Watermelon' |
| _(null)_ | _(null)_ | 4 | 'Pear' |

Правое соединение возвращает все данные из правой таблицы. Если есть совпадения в левой - они обогащаюют данные. Иначе - вместо них NULL.


![](PostgreSQL-Join-Right-Join.png)

**Кстати**, чтобы получить только те строки, которые не содержат данных в левой таблице можно использовать оператор Where:

```sql
SELECT
    a.id id_a,
    a.fruit fruit_a,
    b.id id_b,
    b.fruit fruit_b
FROM
    basket_a a
RIGHT JOIN basket_b b ON a.fruit = b.fruit
WHERE a.id IS NULL;
```

| id_a | fruit_d | id_a | fruit_d |
|---|---|---|---|
| _(null)_ | _(null)_ | 3 | 'Watermelon' |
| _(null)_ | _(null)_ | 4 | 'Pear' |


Диаграмма Вена для соответствующего запроса:

![](PostgreSQL-Join-Right-Join-with-Where.png)

## PostgreSQL full outer join
Аналогично можно получить сопоставление по всем строкам в обоих таблицах. Для этого используется оператор **OUTER JOIN**:

```sql
SELECT
    a.id id_a,
    a.fruit fruit_a,
    b.id id_b,
    b.fruit fruit_b
FROM
    basket_a a
FULL OUTER JOIN basket_b b ON a.fruit = b.fruit;
```

| id_a | fruit_d | id_a | fruit_d |
|---|---|---|---|
| 1 | 'Apple' | 2 | 'Apple' |
| 2 | 'Orange' | 1 | 'Orange' |
| 3 | 'Banana' | _(null)_ | _(null)_ |
| 4 | 'Cucumber' | _(null)_ | _(null)_ |
| _(null)_ | _(null)_ | 3 | 'Watermelon' |
| _(null)_ | _(null)_ | 4 | 'Pear' |

![](PostgreSQL-Join-Full-Outer-Join.png)

Чтобы получить список уникальных строк из обоих таблиц, можно так же воспользоваться оператором WHERE:

```sql
SELECT
    a.id id_a,
    a.fruit fruit_a,
    b.id id_b,
    b.fruit fruit_b
FROM
    basket_a a
FULL JOIN basket_b b ON a.fruit = b.fruit
 WHERE a.id IS NULL OR b.id IS NULL;
```

| id_a | fruit_d | id_a | fruit_d |
|---|---|---|---|
| 3 | 'Banana' | _(null)_ | _(null)_ |
| 4 | 'Cucumber' | _(null)_ | _(null)_ |
| _(null)_ | _(null)_ | 3 | 'Watermelon' |
| _(null)_ | _(null)_ | 4 | 'Pear' |

![](PostgreSQL-Join-Full-Outer-Join-with-Where.png)

## Практика 1 (dvdrental)
1. Выведите список названий всех фильмов и их языков (таблица language)
2. Выведите список всех актеров, снимавшихся в фильме Lambs Cincinatti (film_id = 508). Надо использовать 2 join'а и один where 

## Агрегирующие функции

До сих пор мы занимались простыми выборками из БД. Для задач аналитики и машинного обучения требуется создавать на основе выборок агрегаты -
данные группируются по ключу (в качестве ключа выступает один или несколько атрибутов) и внутри каждой группы вычисляются некоторые статистики.


### SUM

Простое суммирование, в качестве аргемента принимает имя колонки

Примечание: признак должен быть числовой, иначе результаты могут быть странные

<pre>
SELECT SUM(rating) FROM public.ratings;
</pre>

Результат:
<pre>
   sum
----------
 91816043
(1 row)
</pre>

### COUNT

Простой счётчик записей. ЕСли передать модификатор DISTINCT - получим только уникальные записи

<pre>
SELECT
    COUNT(userId) as count,
    COUNT(DISTINCT userId) as count_distinct,
    COUNT(DISTINCT userId)/CAST(COUNT(userId) as float) unique_fraction
FROM public.ratings;
</pre>

Результат:
<pre>
 count  | count_distinct |  unique_fraction
--------+----------------+--------------------
 777776 |           7956 | 0.0102291662380943
(1 row)
</pre>

Несколько особенностей запроса

* несколько аггрегатов в одной строке
* использовали alias - дали имя колонке
* применили арифметическую операцию к результатам запроса (деление) - посчитали отношение уникальных userId к общему числу записей.

### AVG

AVG (AVERAGE) - вычисление среднего значения

<pre>
SELECT AVG(rating) from public.ratings;
</pre>

Результат:
<pre>
       avg
------------------
 3.52809035436088
(1 row)
</pre>




## Практика 2.
1. Подсчитайте количество актеров в фильме Grosse Wonderful (id - 384)

## Базовые статистики по группам: GROUP BY

Кроме расчёта статистик по всей таблице можно считать значения статистик внутри групп, с помощью аггрегирующего оператора GROUP BY:

Например, можем найти самых активных пользователей - тех, кто поставил больше всего оценок

<pre>
SELECT
    userId,
    COUNT(rating) as activity
FROM public.ratings
GROUP BY userId
ORDER BY activity DESC
LIMIT 5;
</pre>

Результат:
<pre>
 userid | activity
--------+----------

  45811 |    18276
   8659 |     9279
 270123 |     7638
 179792 |     7515
 228291 |     7410
(5 rows)
</pre>

Группировать можно по нескольким полям

<pre>
SELECT
    userId,
    to_char(to_timestamp(timestamp), 'YYYY/MM/DD') as dt,
    COUNT(rating) as activity
FROM public.ratings
GROUP BY 1,2
ORDER BY activity
DESC LIMIT 5;
</pre>

Результат:
<pre>
 userid |     dt     | activity
--------+------------+----------
 270123 | 2015/07/05 |     6663
  45811 | 2015/12/15 |     5085
  24025 | 2016/03/27 |     4946
 101276 | 2016/05/09 |     4834
 258253 | 2017/02/10 |     4227
(5 rows)
</pre>

## Фильтрация: HAVING

Аналогично WHERE оператор HAVING позволяет проводить фильтрацию. Разница том, что фильтруются поля с агрегирующими функциями

<pre>
SELECT
    userId,
    AVG(rating) as avg_rating
FROM public.ratings
GROUP BY userId
HAVING AVG(rating) < 3.5
LIMIT 5;
</pre>

Результат:
<pre>
 userid |    avg_rating
--------+------------------
   5761 | 3.41922005571031
   5468 | 1.66666666666667
   7662 | 3.26373626373626
   4326 | 3.33783783783784
   2466 |           3.4375
(5 rows)
</pre>

## Практика 3
Выведите список фильмов, в которых снималось больше 10 актеров

##  Подзапросы

Cпособ разделения логики формирования выборки - подзапросы. Подзапрос - это SELECT, результаты которого используются в другом SELECT/

<pre>
SELECT DISTINCT
    userId
FROM public.ratings
WHERE
    rating < (
            SELECT AVG(rating)
            FROM public.ratings
    )
LIMIT 5;
</pre>

Результат:
<pre>
 userid
--------
 233338
 174416
 196916
 164125
 157514
(5 rows)
</pre>

## Оконные(аналитические) функции

Оконные функции - полезный инструмент для построения сложных аналитических запросов.

Для их использования нужно задать парметры окна и функцию, которую хотим посчитать на каждом объекте внутри окна.

Простой пример - функция ROW_NUMBER(). Эта функция нумерует строки внутри окна. Пронумеруем контент для каждого пользователя в порядке убывания рейтингов.

<pre>
SELECT
  userId, movieId, rating,
  ROW_NUMBER() OVER (PARTITION BY userId ORDER BY rating DESC) as movie_rank
FROM (
    SELECT DISTINCT
        userId, movieId, rating
    FROM ratings
    WHERE userId <>1 LIMIT 1000
) as sample
ORDER BY
    userId,
    rating DESC,
    movie_rank
LIMIT 20;
</pre>

Результат:
<pre>
userid | movieid | rating | movie_rank
--------+---------+--------+------------
      2 |    1356 |      5 |          1
      2 |     339 |      5 |          2
      2 |     648 |      4 |          3
      2 |     605 |      4 |          4
      2 |    1233 |      4 |          5
      2 |    1210 |      4 |          6
      2 |     377 |      4 |          7
      2 |     260 |      4 |          8
      2 |      79 |      4 |          9
      2 |     628 |      4 |         10
      2 |      64 |      4 |         11
      2 |      58 |      3 |         12
      2 |      25 |      3 |         13
      2 |     762 |      3 |         14
</pre>

Параметры запроса:

* ROW_NUMBER - функция, которую применяем к окну
* OVER - описание окна

Описание окна содержит:
* PARTITION BY - поле (или список полей), которые описывают группу строк для применения оконной функции
* ORDER BY - поле, которое задаёт порядок записей внутри окна. Для полей внутри ORDER BY можно применять стандартные модификаторы DESC, ASC

Оконнная функция никак не меняет количество строк в выдаче, но к каждой строке добавляется полезная информация - например, про порядковый номер строки внутри окна

Названия функций обычно отражают их ссмысл. Ниже будут прриведены примеры использования и результаты запросо

### SUM()

Суммирует значения внутри окна. Посчитаем странную метрику - разделим каждое значение рейтинга на сумму всех рейтингов этого пользователя.

<pre>
SELECT userId, movieId, rating,
    rating / SUM(rating) OVER (PARTITION BY userId) as strange_rating_metric
FROM (SELECT DISTINCT userId, movieId, rating FROM ratings WHERE userId <>1 LIMIT 1000) as sample
ORDER BY userId, rating DESC
LIMIT 20;
</pre>

Результат:
<pre>
 userid | movieid | rating | strange_rating_metric
--------+---------+--------+-----------------------
      2 |     339 |      5 |    0.0684931506849315
      2 |    1356 |      5 |    0.0684931506849315
      2 |     648 |      4 |    0.0547945205479452
      2 |      64 |      4 |    0.0547945205479452
      2 |      79 |      4 |    0.0547945205479452
      2 |     260 |      4 |    0.0547945205479452
      2 |    1233 |      4 |    0.0547945205479452
      2 |    1210 |      4 |    0.0547945205479452
      2 |     377 |      4 |    0.0547945205479452
      2 |     605 |      4 |    0.0547945205479452
      2 |     628 |      4 |    0.0547945205479452
      2 |     762 |      3 |    0.0410958904109589
      2 |     141 |      3 |    0.0410958904109589
      2 |     780 |      3 |    0.0410958904109589
      2 |       5 |      3 |    0.0410958904109589
      2 |      58 |      3 |    0.0410958904109589
      2 |      25 |      3 |    0.0410958904109589
      2 |    1475 |      3 |    0.0410958904109589
      2 |      32 |      2 |    0.0273972602739726
      2 |    1552 |      2 |    0.0273972602739726
(20 rows)
</pre>

### COUNT(), AVG()

Счётчик элементов внутри окна, а так же функция Average(). Для наглядности воспользуемся ими одновременно - результаты не должны отличаться.
Вычислим полезную метрику - отклонение рейтинга пользователя от среднего рейтинга, который он склонен выставлять

<pre>
SELECT userId, movieId, rating,
    rating - AVG(rating) OVER (PARTITION BY userId) rating_deviance_simplex,
    rating - SUM(rating) OVER (PARTITION BY userId) /COUNT(rating) OVER (PARTITION BY userId) as rating_deviance_complex
FROM (SELECT DISTINCT userId, movieId, rating FROM ratings WHERE userId <>1 LIMIT 1000) as sample
ORDER BY userId, rating DESC
LIMIT 20;
</pre>

Результат:
<pre>
 userid | movieid | rating | rating_deviance_simplex | rating_deviance_complex
--------+---------+--------+-------------------------+-------------------------
      2 |     339 |      5 |        1.68181818181818 |        1.68181818181818
      2 |    1356 |      5 |        1.68181818181818 |        1.68181818181818
      2 |     648 |      4 |       0.681818181818182 |       0.681818181818182
      2 |      64 |      4 |       0.681818181818182 |       0.681818181818182
      2 |      79 |      4 |       0.681818181818182 |       0.681818181818182
      2 |     260 |      4 |       0.681818181818182 |       0.681818181818182
      2 |    1233 |      4 |       0.681818181818182 |       0.681818181818182
      2 |    1210 |      4 |       0.681818181818182 |       0.681818181818182
      2 |     377 |      4 |       0.681818181818182 |       0.681818181818182
      2 |     605 |      4 |       0.681818181818182 |       0.681818181818182
      2 |     628 |      4 |       0.681818181818182 |       0.681818181818182

</pre>

## Практика 4
Выведите таблицу с 3-мя полями: название фильма, имя актера и количества фильмов, в которых он снимался

## Домашнее задание (по базе dvdrental)
1. Выведите магазины, имеющие больше 300-от покупателей 
2. Сделайте запрос к таблице rental. Добавьте колонку с порядковым номером аренды (сортировать по rental_date) для каждого юзера.

## Полезные материалы

- https://habr.com/ru/post/268983/
- http://www.postgresqltutorial.com/postgresql-aggregate-functions/
- http://www.skillz.ru/dev/php/article-Obyasnenie_SQL_obedinenii_JOIN_INNER_OUTER.html